retrieve weather data: one file per month per wpp (allegedly more efficient, but not the case : 12 files à 36 min instead of 1 file à 5 h)

In [ ]:
import json
import datetime
import os
import shutil
from ecmwfapi import ECMWFService

# Load JSON Data
with open("../data/WPPs+production_new.json", "r", encoding="utf-8") as f:
    wpps = json.load(f)

# ECMWF API Connection
server = ECMWFService("mars")

# Create a temporary directory for GRIB files
temp_dir = "reforecast_data"
os.makedirs(temp_dir, exist_ok=True)

# Function to get unique months from the production data
def get_unique_months(production_data):
    months = set()
    for entry in production_data:
        timestamp = datetime.datetime.fromisoformat(entry[0])
        months.add((timestamp.year, timestamp.month))
    return sorted(months)

# Dictionary to track already downloaded forecasts by (lat, lon, year, month)
downloaded_forecasts = {}

# Process each wind farm
for i, wpp in enumerate(wpps):
    lat, lon = wpp["Latitude"], wpp["Longitude"]
    json_id = wpp["JSON-ID"]

    # Get unique months for which production data exists
    unique_months = get_unique_months(wpp["Production"])

    print(f"Processing wind forecast data for: {wpp['Name']} ({lat}, {lon})...")

    # Retrieve forecasts for each month
    for year, month in unique_months:
        # Define the start and end dates for the month
        start_date = (datetime.date(year, month, 1) - datetime.timedelta(days=6)).strftime("%Y-%m-%d")
        end_date = (datetime.date(year, month + 1, 1) - datetime.timedelta(days=1)).strftime("%Y-%m-%d") if month != 12 else (datetime.date(year + 1, 1, 1) - datetime.timedelta(days=1)).strftime("%Y-%m-%d")

        # Generate file name
        new_grib_file = os.path.join(temp_dir, f"{i}_{json_id}_{year}_{month:02d}.grib")

        # **Check if the file already exists**
        if os.path.exists(new_grib_file):
            print(f"File {new_grib_file} already exists. Skipping download.")
            downloaded_forecasts[(lat, lon, year, month)] = new_grib_file
            continue

        # **Check if another WPP at the same location has already downloaded this month's forecast**
        location_key = (lat, lon, year, month)
        if location_key in downloaded_forecasts:
            existing_grib_file = downloaded_forecasts[location_key]
            shutil.copy(existing_grib_file, new_grib_file)
            print(f"Copied existing forecast {existing_grib_file} to {new_grib_file}.")
            continue

        # **Download the forecast (only if it's not already available)**
        print(f"Retrieving data for {year}-{month:02d} for {wpp['Name']} ({lat}, {lon})...")
        server.execute(
            {
                "class": "od",
                "expver": "1",
                "stream": "oper",
                "type": "fc",
                "levtype": "sfc",
                "param": "246.228/247.228",  # u and v wind components at 100m
                "date": f"{start_date}/to/{end_date}",
                "time": "00/12",
                "step": "0/1/2/3/4/5/6/7/8/9/10/11/12/13/14/15/16/17/18/19/20/21/22/23/24/25/26/27/28/29/30/31/32/33/34/35/36/37/38/39/40/41/42/43/44/45/46/47/48/49/50/51/52/53/54/55/56/57/58/59/60/61/62/63/64/65/66/67/68/69/70/71/72/73/74/75/76/77/78/79/80/81/82/83/84/85/86/87/88/89/90/93/96/99/102/105/108/111/114/117/120/123/126/129/132/135/138/141/144",  # Forecasts up to 6 days ahead
                "area": f"{lat}/{lon}/{lat}/{lon}",
                "grid": "0.25/0.25",
                "format": "grib2"
            },
            target=new_grib_file  # Save the GRIB file to a temporary location
        )

        print(f"Retrieved data for {year}-{month:02d} and saved to {new_grib_file}.")
        downloaded_forecasts[location_key] = new_grib_file

print("All forecasts processed.")


Processing wind forecast data for: A1 DE PARC EOLIEN EN MER DE LA BAIE DE ST-BRIEUC (48.853, -2.539)...
File reforecast_data\0_168_2023_11.grib already exists. Skipping download.
File reforecast_data\0_168_2023_12.grib already exists. Skipping download.
File reforecast_data\0_168_2024_01.grib already exists. Skipping download.
File reforecast_data\0_168_2024_02.grib already exists. Skipping download.
File reforecast_data\0_168_2024_03.grib already exists. Skipping download.
File reforecast_data\0_168_2024_04.grib already exists. Skipping download.
File reforecast_data\0_168_2024_05.grib already exists. Skipping download.
File reforecast_data\0_168_2024_06.grib already exists. Skipping download.
File reforecast_data\0_168_2024_07.grib already exists. Skipping download.
File reforecast_data\0_168_2024_08.grib already exists. Skipping download.
File reforecast_data\0_168_2024_09.grib already exists. Skipping download.
File reforecast_data\0_168_2024_10.grib already exists. Skipping downlo